In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import nltk
import string

from sklearn.model_selection import learning_curve


from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn.model_selection import train_test_split

## Read and Prep Data


In [ ]:
tqdm.pandas()

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive') 
path = '/content/gdrive/MyDrive/IFT 6390/kaggle2/'

Mounted at /content/gdrive


In [ ]:
input_path = path+'data/input/'

In [ ]:
all_train_data = pd.read_csv(input_path + 'train_data_prepped.csv').fillna('')
all_test_data = pd.read_csv(input_path + 'test_data_prepped.csv').fillna('')

In [ ]:
all_train_data.head(1)

,id,text,cat,text_no_punc,text_no_numerals,text_no_sw,text_porter_stemmed,text_lancaster_stemmed,text_lemmatized
0,0,anyway im getting of for a while,train,anyway im getting of for a while,anyway im getting of for a while,anyway im getting,anyway im get of for a while,anyway im get of for a whil,anyway im getting of for a while


In [ ]:
all_test_data.head(1)

,id,text,cat,text_no_punc,text_no_numerals,text_no_sw,text_porter_stemmed,text_lancaster_stemmed,text_lemmatized
0,0,so cut... missed out on all the tce extreme me...,test,so cut missed out on all the tce extreme merch...,so cut missed out on all the tce extreme merch...,cut missed tce extreme merch! sizes,so cut miss out on all the tce extrem merch! i...,so cut miss out on al the tce extrem merch! in...,so cut missed out on all the tce extreme merch...


In [ ]:
all_train_data[all_train_data.isnull().any(axis=1)]

,id,text,cat,text_no_punc,text_no_numerals,text_no_sw,text_porter_stemmed,text_lancaster_stemmed,text_lemmatized


In [ ]:
all_test_data[all_test_data.isnull().any(axis=1)]

,id,text,cat,text_no_punc,text_no_numerals,text_no_sw,text_porter_stemmed,text_lancaster_stemmed,text_lemmatized


load train targets, binarize, drop nuetrals

In [ ]:
train_target = pd.read_csv(input_path + 'train_results.csv')
train_target['target'].value_counts()

positive    520436
negative    519803
neutral         84
Name: target, dtype: int64

In [ ]:
train_target.head()

,id,target
0,0,positive
1,1,negative
2,2,positive
3,3,positive
4,4,negative


In [ ]:
train_target['target'] = [0 if t == 'negative' else 2 if t == 'positive' else 2 for t in train_target['target'].values]

In [ ]:
all_train_data['target'] = train_target['target']

drop nuetral target rows to keep it binary

In [ ]:
# all_train_data = all_train_data[all_train_data['target'] != 'neutral']
# all_train_data['target'] = [1 if t=='positive' else 0 for t in all_train_data['target'].values]
all_train_data['target'].value_counts()


2    520520
0    519803
Name: target, dtype: int64

### prepare bag of words<br>
we need a different model for each text format

Naive Bayes doesn't have any hyperparameters, so we'll test different versions of the text processing with different versions of naive bayes and corpos generation

the following parameters will be tested<br>
1. size of corpus vocab
2. corpus ngram varieties
3. bernoulli, multinomial or gaussian naive bayes
4. stemming/lemmatizing methods

In [ ]:
vocab_sizes = [128, 1024, 8096, 16384]
ngrams = [(1, 1), (2, 2), (1, 2), (1, 3), (1, 4)]
models = [BernoulliNB(), GaussianNB(), MultinomialNB()]
text_columns = ['text_no_numerals', 'text_no_sw', 'text_porter_stemmed', 'text_lancaster_stemmed', 'text_lemmatized']
threshold = 0.5

# shuffle train data
all_train_data = all_train_data.sample(len(all_train_data))

# get data and target
train_X = all_train_data.iloc[:, :-1]
train_y  = all_train_data.iloc[:, -1]


# get 5000 data points for first round of training
train_v1 = train_X[:10_000]
targets_v1 = train_y[:10_000]

In [ ]:
targets_v1.head()

248551    0
974374    2
400458    0
207770    0
1558      0
Name: target, dtype: int64

In [ ]:
def trainModels(df, targets, vocab_sizes, models, text_columns, ngrams, threshold):

    param_tracker = []
    counter = 1
    for size in tqdm(vocab_sizes):
#         print(f'vocab size: {size}')

        for model in models:
#             print(f'model type: {model}')

            for col in text_columns:
#                 print(f'processing method: {col}')


                for ng in tqdm(ngrams):
#                     print(f'ngram range: {ng}')

                    counter+=1
                    if counter % 10==0:
                        print(f'{counter}')
                    vectorizer = CountVectorizer(max_features=size, ngram_range=ng)

#                     print('creating bag of words')
                    BOW = vectorizer.fit_transform(df[col])
                    BOW_array = BOW.toarray()

                    X_train, X_valid, y_train, y_valid = train_test_split(BOW_array, targets, test_size=0.2, random_state=42)
#                     print('fitting data')  


                    model.fit(X_train, y_train)


                    prob_predictions = model.predict_proba(X_valid)

                    # gets index of 1 column in prob_predictions
                    pos_predictions = [pred[list(model.classes_).index(2)] for pred in prob_predictions]

                    num_correct_pred = 0
                    for pred, actual in zip(pos_predictions, y_valid):
                        if pred >= threshold:
                            binary_pred = 2
                        else:
                            binary_pred = 0

                        if binary_pred==actual:
                            num_correct_pred+=1
                    accuracy = num_correct_pred / len(y_valid)
                    print(f'accuracy: {accuracy}')
                    print('\n\n')

                    param_tracker.append((size, str(model), col, ng, accuracy))
                    
    param_df = pd.DataFrame(param_tracker, columns = ['vocab_size', 'model_type', 'text_column', 'ngram', 'accuracy_round1'])
    param_df = param_df.sort_values(by='accuracy_round1', ascending=False)
    return param_df

                    
                
    # for vocab size we'll test

In [ ]:
param_df = trainModels(train_v1, targets_v1, vocab_sizes, models, text_columns, ngrams, threshold)

 20%|██        | 1/5 [00:00<00:01,  2.10it/s]

accuracy: 0.664






 40%|████      | 2/5 [00:00<00:01,  2.05it/s]

accuracy: 0.589






 60%|██████    | 3/5 [00:01<00:01,  1.81it/s]

...

accuracy: 0.728



300



 80%|████████  | 4/5 [00:32<00:08,  8.23s/it]

accuracy: 0.732






100%|██████████| 4/4 [15:42<00:00, 235.54s/it]

accuracy: 0.7315





In [ ]:
param_df

,vocab_size,model_type,text_column,ngram,accuracy_round1
203,8096,MultinomialNB(),text_no_numerals,"(1, 3)",0.7445
164,8096,BernoulliNB(),text_porter_stemmed,"(1, 4)",0.7410
214,8096,MultinomialNB(),text_porter_stemmed,"(1, 4)",0.7405
204,8096,MultinomialNB(),text_no_numerals,"(1, 4)",0.7395
243,16384,BernoulliNB(),text_lancaster_stemmed,"(1, 3)",0.7395
...,...,...,...,...,...
265,16384,GaussianNB(),text_lancaster_stemmed,"(1, 1)",0.5580
6,128,BernoulliNB(),text_no_sw,"(2, 2)",0.5510
260,16384,GaussianNB(),text_porter_stemmed,"(1, 1)",0.5500
56,128,MultinomialNB(),text_no_sw,"(2, 2)",0.5475


In [ ]:
param_df

,vocab_size,model_type,text_column,ngram,accuracy_round1
203,8096,MultinomialNB(),text_no_numerals,"(1, 3)",0.7445
164,8096,BernoulliNB(),text_porter_stemmed,"(1, 4)",0.7410
214,8096,MultinomialNB(),text_porter_stemmed,"(1, 4)",0.7405
204,8096,MultinomialNB(),text_no_numerals,"(1, 4)",0.7395
243,16384,BernoulliNB(),text_lancaster_stemmed,"(1, 3)",0.7395
...,...,...,...,...,...
265,16384,GaussianNB(),text_lancaster_stemmed,"(1, 1)",0.5580
6,128,BernoulliNB(),text_no_sw,"(2, 2)",0.5510
260,16384,GaussianNB(),text_porter_stemmed,"(1, 1)",0.5500
56,128,MultinomialNB(),text_no_sw,"(2, 2)",0.5475


In [ ]:
param_df.to_csv(path+'data/output/naive_bayes_params_round1.csv', index=False)

In [ ]:
param_df = pd.read_csv(path+'data/output/naive_bayes_params_round1.csv')

# ngrams get saved as string, convert back to int tuple
param_df['ngram'] = [(int(ng[1]), int(ng[4])) for ng in param_df['ngram'].values]

param_df = param_df.replace({np.nan: None})

In [ ]:
top_params = param_df[:10]

based on these results, we'll train the full dataset on these 10 combinations

In [ ]:
num_points = 100_000
num_train = int(num_points*0.8)

In [ ]:

model_accuracy = []

for i, param_set in enumerate(tqdm(top_params.values)):
    vocab_size, model_type, text_column, ngram, *_ = param_set
    print(vocab_size, model_type, text_column, ngram,)
    
    print('splitting train-validation data')
    
    # train is all training data with targets included!
    # sample to get smaller amount of data to deal with RAM constraints
    train_sample = all_train_data.sample(num_points)

    # get data and target
    sample_train_X = train_sample.iloc[:, :-1]
    sample_train_y  = train_sample.iloc[:, -1]


    

   
    
#     print(len(X_train), len(y_train), len(X_valid), len(y_valid))
    
    if vocab_size:
        vocab_size = int(vocab_size)
    
    if model_type == 'MultinomialNB()':
        model = MultinomialNB()
    elif model_type == 'BernoulliNB()':
        model = BernoulliNB()
    elif model_type == 'GaussianNB()':
        model = GaussianNB()
    else:
        print('warning, unrecognized model!')
        print(i, param_set)
        
    print('vectorizing')
    vectorizer = CountVectorizer(max_features=vocab_size, ngram_range=ngram)

    print('creating BOW')
    
    BOW = vectorizer.fit_transform(sample_train_X[text_column])
    # BOW_array = BOW.toarray()
    

    
    
    # all sample_train_X and sample_train_y are the X, y sample from all of the training data, 
    #+ sample size determined by num_points
    X_train, X_valid, y_train, y_valid = train_test_split(BOW, sample_train_y, test_size=0.2, random_state=42)
    
    print('fitting model')
    model.fit(X_train, y_train)

    print('getting predictions')
    prob_predictions = model.predict_proba(X_valid)

    # gets index of 1 column in prob_predictions
    pos_predictions = [pred[list(model.classes_).index(2)] for pred in prob_predictions]
    
    
    # get accuracy
    num_correct_pred = 0
    for pred, actual in zip(pos_predictions, y_valid):
        if pred >= threshold:
            binary_pred = 2
        else:
            binary_pred = 0

        if binary_pred==actual:
            num_correct_pred+=1
    accuracy = num_correct_pred / len(y_valid)
    print(accuracy)
    print('\n')
    
    model_accuracy.append(accuracy)
    
    

  0%|          | 0/10 [00:00<?, ?it/s]

8096 MultinomialNB() text_no_numerals (1, 4)
splitting train-validation data
vectorizing
creating BOW
fitting model
getting predictions


 10%|█         | 1/10 [00:19<02:58, 19.86s/it]

0.7719


8096 MultinomialNB() text_no_numerals (1, 2)
splitting train-validation data
vectorizing
creating BOW


 20%|██        | 2/10 [00:27<01:40, 12.51s/it]

...

fitting model
getting predictions
0.7641


8096 MultinomialNB() text_porter_stemmed (1, 4)
splitting train-validation data
vectorizing
creating BOW


 90%|█████████ | 9/10 [01:45<00:11, 11.54s/it]

fitting model
getting predictions
0.7728


8096 BernoulliNB() text_porter_stemmed (1, 4)
splitting train-validation data
vectorizing
creating BOW


100%|██████████| 10/10 [02:01<00:00, 12.16s/it]

fitting model
getting predictions
0.76065




In [ ]:
top_params['accuracy_round2'] = model_accuracy

In [ ]:
top_params = top_params.sort_values('accuracy_round2', ascending=False)

In [ ]:
top_params

,vocab_size,model_type,text_column,ngram,accuracy_round1,accuracy_round2
6,8096,MultinomialNB(),text_porter_stemmed,"(1, 3)",0.7390,0.77490
0,8096,MultinomialNB(),text_no_numerals,"(1, 3)",0.7445,0.77415
5,8096,MultinomialNB(),text_no_numerals,"(1, 2)",0.7390,0.77350
2,8096,MultinomialNB(),text_porter_stemmed,"(1, 4)",0.7405,0.77280
3,8096,MultinomialNB(),text_no_numerals,"(1, 4)",0.7395,0.77190
4,16384,BernoulliNB(),text_lancaster_stemmed,"(1, 3)",0.7395,0.76550
7,16384,BernoulliNB(),text_no_numerals,"(1, 4)",0.7380,0.76545
8,8096,BernoulliNB(),text_porter_stemmed,"(1, 2)",0.7380,0.76450
9,8096,BernoulliNB(),text_no_numerals,"(1, 2)",0.7380,0.76410
1,8096,BernoulliNB(),text_porter_stemmed,"(1, 4)",0.7410,0.76065


In [ ]:
top_params.to_csv(path+'data/output/naive_bayes_params_round2.csv', index=False)

## train final model with best parameters

In [ ]:
all_train_data.head(1)

,id,text,cat,text_no_punc,text_no_numerals,text_no_sw,text_porter_stemmed,text_lancaster_stemmed,text_lemmatized,target
0,0,anyway im getting of for a while,train,anyway im getting of for a while,anyway im getting of for a while,anyway im getting,anyway im get of for a while,anyway im get of for a whil,anyway im getting of for a while,2


In [ ]:
# # 8096	BernoulliNB()	text_porter_stemmed	(1, 2)	

# all_train_data = all_train_data.sample(len(all_train_data))


vectorizer = CountVectorizer(max_features=8096, ngram_range=(1, 2))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(all_train_data['text_porter_stemmed'], 
                                                    all_train_data['target'], 
                                                    test_size=0.2, 
                                                    random_state=42)


In [ ]:
train_BOW = vectorizer.fit_transform(X_train)



In [ ]:
test_BOW = vectorizer.transform(X_test)
# test_BOW_array = test_BOW.toarray()

In [ ]:
model = BernoulliNB()
model.fit(train_BOW, y_train, test_BOW, y_test)

MultinomialNB()

In [ ]:
model.classes_

array([0, 2])

In [ ]:
predictions = model.predict(test_BOW_array)

# # gets index of 1 column in prob_predictions
# pos_predictions = [pred[list(model.classes_).index(1)] for pred in prob_predictions]


# # get accuracy
# num_correct_pred = 0
# for pred, actual in zip(pos_predictions, y_valid):
#     if pred >= threshold:
#         binary_pred = 1
#     else:
#         binary_pred = 0

#     if binary_pred==actual:
#         num_correct_pred+=1
# accuracy = num_correct_pred / len(y_valid)
# print(accuracy)
# print('\n')

In [ ]:
pdf = pd.DataFrame(predictions).reset_index()
pdf.columns  = ['id', 'target']


In [ ]:
pdf['target'].value_counts()

2    288103
0    272072
Name: target, dtype: int64

In [ ]:
# pdf['target'] = [0 if t == 'negative' else 2 if t=='positive' else 2 for t in pdf['target'].values]

In [ ]:
pdf.to_csv('data/output/first_pred.csv', index=False)

code from https://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit


def plot_learning_curve(
    estimator,
    title,
    X,
    y,
    axes=None,
    ylim=None,
    cv=None,
    n_jobs=None,
    scoring=None,
    train_sizes=np.linspace(0.1, 1.0, 5),
):
    """
    Generate 3 plots: the test and training learning curve, the training
    samples vs fit times curve, the fit times vs score curve.

    Parameters
    ----------
    estimator : estimator instance
        An estimator instance implementing `fit` and `predict` methods which
        will be cloned for each validation.

    title : str
        Title for the chart.

    X : array-like of shape (n_samples, n_features)
        Training vector, where ``n_samples`` is the number of samples and
        ``n_features`` is the number of features.

    y : array-like of shape (n_samples) or (n_samples, n_features)
        Target relative to ``X`` for classification or regression;
        None for unsupervised learning.

    axes : array-like of shape (3,), default=None
        Axes to use for plotting the curves.

    ylim : tuple of shape (2,), default=None
        Defines minimum and maximum y-values plotted, e.g. (ymin, ymax).

    cv : int, cross-validation generator or an iterable, default=None
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:

          - None, to use the default 5-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, default=None
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    scoring : str or callable, default=None
        A str (see model evaluation documentation) or
        a scorer callable object / function with signature
        ``scorer(estimator, X, y)``.

    train_sizes : array-like of shape (n_ticks,)
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the ``dtype`` is float, it is regarded
        as a fraction of the maximum size of the training set (that is
        determined by the selected validation method), i.e. it has to be within
        (0, 1]. Otherwise it is interpreted as absolute sizes of the training
        sets. Note that for classification the number of samples usually have
        to be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    if axes is None:
        _, axes = plt.subplots(1, 3, figsize=(20, 5))

    axes[0].set_title(title)
    if ylim is not None:
        axes[0].set_ylim(*ylim)
    axes[0].set_xlabel("Training examples")
    axes[0].set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = learning_curve(
        estimator,
        X,
        y,
        scoring=scoring,
        cv=cv,
        n_jobs=n_jobs,
        train_sizes=train_sizes,
        return_times=True,
    )
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)

    # Plot learning curve
    axes[0].grid()
    axes[0].fill_between(
        train_sizes,
        train_scores_mean - train_scores_std,
        train_scores_mean + train_scores_std,
        alpha=0.1,
        color="r",
    )
    axes[0].fill_between(
        train_sizes,
        test_scores_mean - test_scores_std,
        test_scores_mean + test_scores_std,
        alpha=0.1,
        color="g",
    )
    axes[0].plot(
        train_sizes, train_scores_mean, "o-", color="r", label="Training score"
    )
    axes[0].plot(
        train_sizes, test_scores_mean, "o-", color="g", label="Cross-validation score"
    )
    axes[0].legend(loc="best")

    # Plot n_samples vs fit_times
    axes[1].grid()
    axes[1].plot(train_sizes, fit_times_mean, "o-")
    axes[1].fill_between(
        train_sizes,
        fit_times_mean - fit_times_std,
        fit_times_mean + fit_times_std,
        alpha=0.1,
    )
    axes[1].set_xlabel("Training examples")
    axes[1].set_ylabel("fit_times")
    axes[1].set_title("Scalability of the model")

    # Plot fit_time vs score
    fit_time_argsort = fit_times_mean.argsort()
    fit_time_sorted = fit_times_mean[fit_time_argsort]
    test_scores_mean_sorted = test_scores_mean[fit_time_argsort]
    test_scores_std_sorted = test_scores_std[fit_time_argsort]
    axes[2].grid()
    axes[2].plot(fit_time_sorted, test_scores_mean_sorted, "o-")
    axes[2].fill_between(
        fit_time_sorted,
        test_scores_mean_sorted - test_scores_std_sorted,
        test_scores_mean_sorted + test_scores_std_sorted,
        alpha=0.1,
    )
    axes[2].set_xlabel("fit_times")
    axes[2].set_ylabel("Score")
    axes[2].set_title("Performance of the model")

    return plt


fig, axes = plt.subplots(3, 2, figsize=(10, 15))

X, y = load_digits(return_X_y=True)

title = "Learning Curves (Naive Bayes)"
# Cross validation with 50 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=50, test_size=0.2, random_state=0)

estimator = GaussianNB()
plot_learning_curve(
    estimator,
    title,
    X,
    y,
    axes=axes[:, 0],
    ylim=(0.7, 1.01),
    cv=cv,
    n_jobs=4,
    scoring="accuracy",
)

title = r"Learning Curves (SVM, RBF kernel, $\gamma=0.001$)"
# SVC is more expensive so we do a lower number of CV iterations:
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
estimator = SVC(gamma=0.001)
plot_learning_curve(
    estimator, title, X, y, axes=axes[:, 1], ylim=(0.7, 1.01), cv=cv, n_jobs=4
)